In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("insurance_pre.csv")

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset = pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent = dataset[['age', 'bmi', 'children','sex_male','smoker_yes']]

In [8]:
dependent = dataset[["charges"]]

In [9]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(independent, dependent, test_size=0.30, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
x_train

array([[-1.5330973 , -0.40713453, -0.89833872, -0.97676557, -0.50466988],
       [-0.03364163,  0.32855417, -0.89833872, -0.97676557, -0.50466988],
       [ 0.89459283,  2.56690911,  3.25603402, -0.97676557, -0.50466988],
       ...,
       [ 0.03776102, -0.91016269, -0.89833872,  1.02378711, -0.50466988],
       [-1.46169465,  0.76659782, -0.89833872,  1.02378711, -0.50466988],
       [-0.46205754, -1.96596021, -0.06746417, -0.97676557, -0.50466988]])

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
param_grid = {'criterion':['friedman_mse','squared_error','absolute_error','poisson'],
             'max_features':['auto','sqrt','log2'],
             'splitter':['best','random']}
grid = GridSearchCV(DecisionTreeRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'squared_error',
                                       'absolute_error', 'poisson'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [13]:
re=grid.cv_results_
grid_predictions = grid.predict(x_test)

from sklearn.metrics import r2_score
r_score = r2_score(y_test,grid_predictions)
print('The R_score value for best parameter {}:'.format(grid.best_params_),r_score)

The R_score value for best parameter {'criterion': 'friedman_mse', 'max_features': 'log2', 'splitter': 'best'}: 0.7531609674466184


In [14]:
table = pd.DataFrame.from_dict(re)

In [15]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.025605,0.031355,0.008003,0.000002,friedman_mse,auto,best,"{'criterion': 'friedman_mse', 'max_features': ...",0.731473,0.568224,0.754037,0.602583,0.671173,0.665498,0.071616,2
1,0.012805,0.003919,0.004802,0.003921,friedman_mse,auto,random,"{'criterion': 'friedman_mse', 'max_features': ...",0.659375,0.648820,0.620033,0.682213,0.659330,0.653954,0.020169,5
2,0.009603,0.003198,0.019203,0.022401,friedman_mse,sqrt,best,"{'criterion': 'friedman_mse', 'max_features': ...",0.676144,0.503268,0.624554,0.514280,0.518056,0.567260,0.069944,23
3,0.008003,0.000001,0.001600,0.003200,friedman_mse,sqrt,random,"{'criterion': 'friedman_mse', 'max_features': ...",0.651430,0.601904,0.590990,0.385957,0.645493,0.575155,0.097496,21
4,0.008003,0.005062,0.001600,0.003200,friedman_mse,log2,best,"{'criterion': 'friedman_mse', 'max_features': ...",0.718327,0.671375,0.640690,0.655218,0.665622,0.670246,0.026206,1
5,0.006401,0.003200,0.006403,0.003201,friedman_mse,log2,random,"{'criterion': 'friedman_mse', 'max_features': ...",0.679207,0.598771,0.577600,0.524027,0.586892,0.593299,0.049992,16
6,0.009604,0.003200,0.003200,0.003919,squared_error,auto,best,"{'criterion': 'squared_error', 'max_features':...",0.717730,0.527101,0.772623,0.643276,0.663798,0.664905,0.082283,3
7,0.008003,0.000002,0.004801,0.003920,squared_error,auto,random,"{'criterion': 'squared_error', 'max_features':...",0.654917,0.692528,0.667304,0.671929,0.589057,0.655147,0.035199,4
8,0.009606,0.003200,0.003200,0.003919,squared_error,sqrt,best,"{'criterion': 'squared_error', 'max_features':...",0.748994,0.412287,0.647922,0.658371,0.545361,0.602587,0.114971,14
9,0.016004,0.016000,0.004803,0.003921,squared_error,sqrt,random,"{'criterion': 'squared_error', 'max_features':...",0.785830,0.507454,0.650868,0.651033,0.592797,0.637596,0.090856,8


In [16]:
age_input=int(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=int(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Somker Yes 0 or 1:"))

Age:33
BMI:255.68
Children:3
Sex Male 0 or 1:1
Somker Yes 0 or 1:0


In [17]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print('Future_Prediction = {}'.format(Future_Prediction))

Future_Prediction = [16085.1275]
